In [1]:
import meep as mp
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Video
from scipy.constants import pi, c
from scipy.signal import hilbert, find_peaks
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit


In [2]:
plt.rcParams['figure.figsize'] = [10,5]

In [3]:
#Define conversion functions/factors to get to/from meep units and kms
#Length factor to transform lengths in kms
l_unit = 1e-6 #1 um
t_unit = l_unit/c #time unit is length/c

#multiplicative constants to convert frequencies from meep to kms hertz
freq_unit = c/l_unit

print("1 meep Length unit is", l_unit*1e+6, "um")
print("1 meep Time unit is", t_unit*1e+15, "fs")
print("1 meep Frequency unit is", freq_unit*1e-12, "THz")

1 meep Length unit is 1.0 um
1 meep Time unit is 3.3356409519815204 fs
1 meep Frequency unit is 299.792458 THz


# Resolution

In [4]:
#courant factor
S = 0.5

#resolution is number of pixels per meep unit
resolution = 20

#resolutions in meep units
space_res = 1/resolution
time_res = space_res*S

print("Pixel per unit: ", resolution)

print("Space resolution [meep]: ", space_res)
print("Time resolution [meep]: ", time_res)

print("Space resolution [um]: ", space_res*l_unit*1e+6)
print("Time resolution [fs]: ", time_res*t_unit*1e+15)

Pixel per unit:  20
Space resolution [meep]:  0.05
Time resolution [meep]:  0.025
Space resolution [um]:  0.049999999999999996
Time resolution [fs]:  0.08339102379953803


# Geometry Definition
Cell

In [5]:
#define cell size and resolution
#Cell size is in meep units
#center is in (0,0), ranges in x=(-sx/2,sx/2), y=(-sy/2,sy/2)
sx = 24
sy = 20

cell = mp.Vector3(sx, sy)

#boundary conditions thickness
pml_thickness = 1.0

#automatic positioning of boundaries
x_bound_up = sx/2 - pml_thickness
x_bound_low = -sx/2 + pml_thickness

y_bound_up = sy/2 - pml_thickness
y_bound_low = -sy/2 + pml_thickness

# $\Delta \phi$ calculation

In [6]:
run_time = 500

distances = np.arange(0.1,1,0.1)
loop_f_central_resonance = np.ones(distances.shape())*0.4529

Delta_Phi_list = []

TypeError: 'int' object is not callable

In [ ]:
for index,d in enumerate(distances):
    print("###############")
    print("Beginning iteration: ",index)
    print("Distance: ",d)
    print("Frequency: ", loop_f_central_resonance[index])



    #materials refractive indexes
    n_bgnd = 1
    n_wvg = 1.7
    n_ring = n_wvg

    #Width properties
    wvg_width = 1

    ring_r = 4
    ring_w = wvg_width

    #Position properties
    wvg_x = 0
    wvg_y = -(wvg_width + d + ring_w*2 + ring_r*2)/2 + wvg_width/2

    ring_x = 0
    ring_y = wvg_y + wvg_width/2 + d + ring_w + ring_r

    #Soure parameters
    source_x = x_bound_low + 1
    source_y = wvg_y

    source_width = wvg_width


    #Geometry objects
    waveguide = mp.Block(mp.Vector3(mp.inf, wvg_width, mp.inf),
                        center = mp.Vector3(wvg_x, wvg_y),
                        material = mp.Medium(index=n_wvg))
    ext_ring = mp.Cylinder(radius=ring_r+ring_w,
                        center=mp.Vector3(ring_x, ring_y),
                        material = mp.Medium(index=n_ring))
    int_ring = mp.Cylinder(radius=ring_r,
                        center=mp.Vector3(ring_x, ring_y))

    geometry = [waveguide,
                ext_ring,
                int_ring]

    #define sources
    #gaussian pulse with f_central and df
    sources = [mp.Source(mp.ContinuousSource(frequency=loop_f_central_resonance[index], width=20),
                        component=mp.Ez,
                        center=mp.Vector3(source_x, source_y),
                        size=mp.Vector3(0, source_width))]

    #define boundary conditions
    pml_layers = [mp.PML(pml_thickness)]

    #Instance simulation with chosen parameters
    sim = mp.Simulation(cell_size=cell,
                        boundary_layers=pml_layers,
                        sources=sources,
                        geometry=geometry,
                        resolution=resolution)


    #Simulate
    mp.verbosity(0)
    sim.run(until=run_time)

    ez_data_wvg = sim.get_array(center=mp.Vector3(0,wvg_y), size=mp.Vector3(sx, 0.001), component=mp.Ez)
    print(ez_data_wvg.shape)

    x = np.arange(-sx/2, sx/2, 1/resolution)

    init_wave = ez_data_wvg[:int(len(ez_data_wvg)/3)][:,0]
    init_x = x[:int(len(ez_data_wvg)/3)]

    fin_wave = ez_data_wvg[int(2*len(ez_data_wvg)/3):][:,0]
    fin_x = x[int(2*len(ez_data_wvg)/3):]

    init_peaks, _ = find_peaks(init_wave, height=0.1)
    fin_peaks, _ = find_peaks(fin_wave, height=0.0)

    init_x_peaks = init_x[init_peaks]
    fin_x_peaks = fin_x[fin_peaks]

    def func_linear(m, k, phi):
        return (m*2*pi + pi/2 -phi)/k

    popt_init, pcov_init = curve_fit(func_linear, [i[0] for i in enumerate(init_x_peaks)], init_x_peaks)

    k_fit_init = popt_init[0]
    phi_init = popt_init[1]%(2*pi)-pi

    print("K_init: ",k_fit_init," phi_init:", phi_init)

    popt_fin, pcov_fin = curve_fit(func_linear, [i[0] for i in enumerate(fin_x_peaks)], fin_x_peaks)

    k_fit_fin = popt_fin[0]
    phi_fin = popt_fin[1]%(2*pi)-pi

    print("K_fin: ",k_fit_fin," phi_fin:", phi_fin)


    Delta_Phi = phi_fin - phi_init
    print("Delta Phi:", Delta_Phi, ", Degrees: ",Delta_Phi*360/(2*pi))
    
    Delta_Phi_list = np.append(Delta_Phi_list, Delta_Phi)




In [ ]:
plt.plot(distances, Delta_Phi_list, "r*")

plt.xlabel(r"Distance d [meep units]")
plt.ylabel(r"$\Delta \Phi$")
plt.grid()